# **Etapa 4: Armazenamento Final dos Dados Preparados**
Objetivo da Etapa: Finalizar o processamento dos arquivos textuais extraídos e limpos, organizando-os de forma sistemática e verificando a qualidade do resultado. Esta etapa garante que o corpus final do projeto IAVS esteja estruturado, nomeado corretamente e com conteúdo confiável para uso posterior em modelagem e análise.

In [15]:
#instalando biblioteca e carregando pacotes 
%pip install language-tool-python
import os
import re
import csv
import difflib
import random
from pathlib import Path
import time
import language_tool_python
import subprocess
import unicodedata
import shutil

Note: you may need to restart the kernel to use updated packages.


# Organização, Verificação e Amostragem dos Textos Limpos

Este script realiza as etapas finais do processamento dos arquivos textuais utilizados no projeto IAVS, a partir da pasta textos_limpos. Ele executa três ações principais: organização por doença/tema, verificação da padronização dos nomes dos arquivos, e revisão amostral do conteúdo.

- **Etapa 4.1 – Organização dos Arquivos**
Os arquivos .txt são lidos da pasta textos_limpos. Com base no prefixo de seus nomes (antes do primeiro sublinhado), é inferido o tema ou a doença. O script então cria subpastas com esse nome dentro da pasta corpus_final e copia os arquivos para seus respectivos destinos. Isso garante que o corpus final esteja organizado de forma lógica e temática, o que facilita a navegação, o versionamento e o uso posterior em análises.

 - **Etapa 4.2 – Verificação da Estrutura**
Após a cópia, o script verifica se as subpastas foram devidamente criadas na pasta corpus_final. Se não houver nenhuma subpasta, um alerta é exibido. Caso existam, seus nomes são listados como confirmação da estrutura.

- **Etapa 4.3 – Verificação de Nomenclatura e Conteúdo**
Duas verificações adicionais são realizadas:

- **Padrão de nomes dos arquivos:**
O script utiliza uma expressão regular para verificar se os nomes seguem o formato:
doenca_tipo_origem2023.txt, com letras minúsculas, underscores como separadores e ano com quatro dígitos ao final.
Arquivos fora do padrão são listados com alerta.

- **Amostragem de conteúdo:**
Três subpastas são selecionadas aleatoriamente. Em cada uma, um arquivo é escolhido ao acaso e os primeiros 800 caracteres do seu conteúdo são exibidos. Essa leitura rápida serve como revisão qualitativa para identificar possíveis erros de extração ou formatação residual.


In [25]:
# ======================== CONFIGURAÇÃO DOS CAMINHOS ======================== #
PASTA_LIMPOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_limpos"
PASTA_FINAL = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"

# Cria a pasta de destino final se ainda não existir
os.makedirs(PASTA_FINAL, exist_ok=True)

# ======================== ETAPA 4.1 – ORGANIZAÇÃO ======================== #
print("\n Iniciando organização dos arquivos...\n")

# Percorre todos os arquivos da pasta de textos limpos
for arquivo in os.listdir(PASTA_LIMPOS):
    if arquivo.endswith(".txt") and "_" in arquivo:
        # Extrai o nome da doença ou tema (prefixo antes do primeiro "_")
        doenca = arquivo.split("_")[0].lower()
        
        # Cria a subpasta correspondente à doença, se não existir
        destino_pasta = os.path.join(PASTA_FINAL, doenca)
        os.makedirs(destino_pasta, exist_ok=True)

        # Define caminhos de origem e destino
        origem_arquivo = os.path.join(PASTA_LIMPOS, arquivo)
        destino_arquivo = os.path.join(destino_pasta, arquivo)

        # Copia o arquivo para a subpasta correta
        shutil.copy2(origem_arquivo, destino_arquivo)
        print(f"✅ Copiado: {arquivo} → {doenca}/")

print("\n Organização concluída.\n")

# ======================== ETAPA 4.2 e 4.3 – VERIFICAÇÃO ======================== #

# Verificação 1: lista as subpastas criadas
print(" Subpastas encontradas em corpus_final:")
subpastas = [nome for nome in os.listdir(PASTA_FINAL) if os.path.isdir(os.path.join(PASTA_FINAL, nome))]
if not subpastas:
    print("⚠️ Nenhuma subpasta encontrada. A organização não foi realizada corretamente.")
else:
    for sub in subpastas:
        print(f"  ✅ {sub}")

# Verificação 2: checagem do padrão de nomes dos arquivos
print("\n Verificando padrão de nomes dos arquivos...")
padrao_nome = re.compile(r"^[a-z0-9]+_[a-z0-9]+_[a-z0-9]+[0-9]{4}\.txt$")
erros_nome = []

# Percorre cada subpasta para validar os nomes dos arquivos
for sub in subpastas:
    caminho_sub = os.path.join(PASTA_FINAL, sub)
    for arquivo in os.listdir(caminho_sub):
        if not padrao_nome.match(arquivo):
            erros_nome.append((sub, arquivo))

# Lista os arquivos que não seguem o padrão
if erros_nome:
    print("\n Arquivos com nome fora do padrão:")
    for sub, nome in erros_nome:
        print(f"  ❌ {nome} (subpasta: {sub})")
else:
    print("✅ Todos os arquivos seguem o padrão de nomenclatura.")

# Verificação 3: leitura amostral de conteúdo para revisão manual
print("\n Amostras de conteúdo para revisão:")

# Seleciona aleatoriamente até 3 subpastas
for sub in random.sample(subpastas, min(3, len(subpastas))):
    caminho_sub = os.path.join(PASTA_FINAL, sub)
    arquivos_txt = [f for f in os.listdir(caminho_sub) if f.endswith(".txt")]
    
    if arquivos_txt:
        # Escolhe aleatoriamente um arquivo para exibição
        amostra = random.choice(arquivos_txt)
        caminho_arquivo = os.path.join(caminho_sub, amostra)
        
        print(f"\n Arquivo: {amostra} (em {sub})")
        with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as f:
            conteudo = f.read()
            print(conteudo[:800])  # Exibe os primeiros 800 caracteres para revisão


 Iniciando organização dos arquivos...

✅ Copiado: adenovírus_guia_1.txt → adenovírus/
✅ Copiado: arboviroses_nota_técnica_1.txt → arboviroses/
✅ Copiado: cancro_mole_protocolo_1.txt → cancro/
✅ Copiado: chikungunya_nota_técnica_1.txt → chikungunya/
✅ Copiado: chikungunya_protocolo_1.txt → chikungunya/
✅ Copiado: chikungunya_protocolo_2.txt → chikungunya/
✅ Copiado: chikungunya_protocolo_3.txt → chikungunya/
✅ Copiado: clamídia_protocolo_1.txt → clamídia/
✅ Copiado: coinfecção_com_hepatite_b_nota_técnica_1.txt → coinfecção/
✅ Copiado: coqueluche_nota_técnica_1.txt → coqueluche/
✅ Copiado: coqueluche_nota_técnica_2.txt → coqueluche/
✅ Copiado: coqueluche_nota_técnica_3.txt → coqueluche/
✅ Copiado: covid-19_guia_1.txt → covid-19/
✅ Copiado: covid-19_nota_técnica_1.txt → covid-19/
✅ Copiado: cólera_nota_técnica_1.txt → cólera/
✅ Copiado: cólera_protocolo_1.txt → cólera/
✅ Copiado: dengue_nota_técnica_1.txt → dengue/
✅ Copiado: dengue_protocolo_1.txt → dengue/
✅ Copiado: dengue_protocolo_

# Etapa de Padronização de Nomes
**Objetivo:**
Padronizar os nomes dos arquivos textuais organizados na pasta corpus_final, garantindo consistência, legibilidade e alinhamento com os critérios estabelecidos para o projeto IAVS.

**O que o script faz:**
- Converte todos os nomes para letras minúsculas.

-  Remove acentuação, parênteses, hífens, espaços e símbolos especiais.

- Substitui por underscore onde necessário.

- Identifica o tipo do documento com base no nome (guia, nota_tecnica, manual, protocolo).

- Define o sufixo padronizado, incluindo o tipo do documento e o ano (_guiavsa2023.txt, _manual2023.txt, etc.).

- Preserva número final, se houver (_1, _2), ao reconstruir o nome limpo.

- Renomeia os arquivos somente se o nome novo for diferente do atual.


**Exemplo de renomeação:**
Um arquivo com nome original:

Guia de vigilância (Volume 1) - versão final.TXT

Será renomeado para:
vigilancia_guiavsa2023.txt

In [26]:
# Caminho da pasta final onde os textos já estão organizados por doença
PASTA_FINAL = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"

# --------------------------- FUNÇÃO DE PADRONIZAÇÃO --------------------------- #
def padronizar_nome(nome_antigo):
    """
    Padroniza o nome de um arquivo textual com base nas regras do projeto:
    - Converte para minúsculas
    - Remove acentos e símbolos
    - Substitui hífens/espaços por underscores
    - Define o sufixo do nome com base no tipo do documento (guia, manual etc.)
    - Adiciona o ano (2023) e o tipo do documento ao final
    """

    # Converte tudo para minúsculas
    nome_original = nome_antigo.lower()

    # 1. Remove acentuação
    nome = unicodedata.normalize('NFKD', nome_original).encode('ASCII', 'ignore').decode('utf-8')

    # 2. Remove parênteses e símbolos especiais
    nome = re.sub(r'[^\w\s\-]', '', nome)

    # 3. Substitui hífens e espaços por underscore
    nome = nome.replace('-', '_').replace(' ', '_')

    # 4. Remove múltiplos underscores consecutivos
    nome = re.sub(r'_+', '_', nome)

    # 5. Remove extensão para trabalhar com o nome-base
    nome_base = nome.replace('.txt', '')

    # 6. Separa o nome por partes para futura lógica de reconstrução
    partes = nome_base.split('_')
    
    # 7. Define tipo de documento e sufixo padrão
    tipos = ['guia', 'nota_tecnica', 'manual', 'protocolo']
    tipo = 'documento'
    sufixo = '_documento2023.txt'

    for t in tipos:
        if t in nome_base:
            tipo = t
            if t == 'guia':
                sufixo = '_guiavsa2023.txt'
            elif t == 'nota_tecnica':
                sufixo = '_notatecnica2023.txt'
            elif t == 'manual':
                sufixo = '_manual2023.txt'
            elif t == 'protocolo':
                sufixo = '_protocolo2023.txt'
            break

    # 8. Verifica se há número ao final do nome (ex: "_1", "_2")
    match = re.search(r'_(\d+)$', nome_base)
    numero = f"_{match.group(1)}" if match else ""

    # 9. Remove o tipo e o número do nome-base original, se existirem
    nome_base_limpo = re.sub(f"_{tipo}{numero}$", '', nome_base)

    # 10. Monta o novo nome padronizado
    novo_nome = f"{nome_base_limpo}{sufixo}"

    return novo_nome

# --------------------------- EXECUÇÃO DA PADRONIZAÇÃO --------------------------- #
print("\n Iniciando padronização dos nomes dos arquivos...\n")

# Percorre cada subpasta (doença) dentro de corpus_final
for subpasta in os.listdir(PASTA_FINAL):
    caminho_sub = os.path.join(PASTA_FINAL, subpasta)
    if os.path.isdir(caminho_sub):
        for arquivo in os.listdir(caminho_sub):
            if arquivo.endswith('.txt'):
                novo_nome = padronizar_nome(arquivo)
                caminho_antigo = os.path.join(caminho_sub, arquivo)
                caminho_novo = os.path.join(caminho_sub, novo_nome)

                # Renomeia o arquivo se houver diferença
                if arquivo != novo_nome:
                 if not os.path.exists(caminho_novo):
                    os.rename(caminho_antigo, caminho_novo)
                    print(f"✅ Renomeado: {arquivo} → {novo_nome}")
                 else:
                    print(f"⚠️ Ignorado (já existe com o nome final): {novo_nome}")



 Iniciando padronização dos nomes dos arquivos...

⚠️ Ignorado (já existe com o nome final): adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023.txt
✅ Renomeado: adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023.txt → adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt
✅ Renomeado: adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt → adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt
✅ Renomeado: adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt → adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt
⚠️ Ignorado (já existe com o nome final): adenovirus_guia_1txt_guiavsa2023.txt
⚠️ Ignorado (já existe com o nome final): arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023.txt
✅ Renomeado: arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023.txt 

# Verificação de trechos aleatórios e consistência dos textos

Este é um verificador inteligente de qualidade textual, voltado para amostragens manuais e automáticas. Ele:

- Lê 2 arquivos aleatórios por subpasta.

-  Em cada arquivo, exibe até 3 trechos aleatórios.

- Checa nesses trechos a presença de padrões indesejáveis como: 
    - "página X de Y"

    - espaços em branco excessivos

    - conteúdo de teste (ex: xxxxx, lorem ipsum)

    - resíduos técnicos ou erros de OCR

    - Informa no terminal se encontrou algo estranho, com o trecho destacado.

In [27]:
# Caminho para a pasta com os textos finais organizados por subpasta (por doença ou tema)
PASTA_FINAL = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"

# Lista de padrões suspeitos que indicam problemas no texto
PADROES_PROBLEMAS = [
    "página [0-9]+ de [0-9]+",         # numeração de página residual
    "confidencial",                   # marcações internas
    "xxxxx",                          # preenchimentos de teste ou censura
    "lorem ipsum",                    # textos fictícios
    "^\s*$",                          # linha em branco
    r'\b(pdf|doc|versao|versão|arquivo)\b',  # termos técnicos que podem indicar resíduos
    r'\.{5,}',                        # sequências longas de pontos (ex: ".....")
]

print("\n INÍCIO DA VERIFICAÇÃO DE TRECHOS ALEATÓRIOS DOS TEXTOS\n")

# ---------------------- Função para revisar um arquivo ---------------------- #
def revisar_arquivo(caminho_arquivo):
    """
    Lê um arquivo e imprime até 3 trechos aleatórios com 5 linhas cada,
    verificando se há padrões suspeitos no conteúdo.
    """
    with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as f:
        linhas = f.readlines()
        total_linhas = len(linhas)

        # Seleciona até 3 trechos aleatórios
        indices = random.sample(range(total_linhas), min(3, total_linhas))
        print(f" {os.path.basename(caminho_arquivo)} — {total_linhas} linhas")

        for i in indices:
            trecho = ''.join(linhas[i:i+5]).strip()
            print(f"\n Trecho aleatório a partir da linha {i+1}:\n{trecho}\n")

            # Verifica se o trecho contém algum padrão problemático
            for padrao in PADROES_PROBLEMAS:
                if re.search(padrao, trecho, re.IGNORECASE):
                    print(f"⚠️ Padrão suspeito detectado: \"{padrao}\"\n")

        print("-" * 80)

# ---------------------- Laço principal: revisão por subpasta ---------------------- #
# Para cada subpasta (tema/doença), seleciona até 2 arquivos para revisão
for subpasta in os.listdir(PASTA_FINAL):
    caminho_sub = os.path.join(PASTA_FINAL, subpasta)
    if os.path.isdir(caminho_sub):
        arquivos_txt = [f for f in os.listdir(caminho_sub) if f.endswith(".txt")]
        amostras = random.sample(arquivos_txt, min(2, len(arquivos_txt)))

        for nome_arquivo in amostras:
            caminho_arquivo = os.path.join(caminho_sub, nome_arquivo)
            revisar_arquivo(caminho_arquivo)

print("\n✅ Verificação finalizada.\n")



 INÍCIO DA VERIFICAÇÃO DE TRECHOS ALEATÓRIOS DOS TEXTOS

 adenovírus_guia_1.txt — 5266 linhas

 Trecho aleatório a partir da linha 4113:
tempo, a confirmação da identificação etiológica se baseia em diagnóstico laboratorial (Brasil, 2023k).
Cabe ressaltar que a vigilância das síndromes gripais no Brasil é voltada para o vírus influenza e o
SARS-CoV-2. A identificação de outros vírus respiratórios, portanto, é realizada por meio de diagnóstico
diferencial para influenza e SARS-CoV-2. Dessa forma, a detecção de OVR ocorre de maneira amostral
e por conveniência, a depender da disponibilidade de insumos disponíveis nas unidades federadas


 Trecho aleatório a partir da linha 3216:
Epidemiológica – Não detectável
Municipal, Estadual,
Distrital H1N1 Yamagata Isolamento* viral
Resultados e
H3N2 Victoria Resistência Caracterização informações


 Trecho aleatório a partir da linha 408:
nacional, também conhecido como Centro Nacional de Influenza (do inglês, National Influenza Center
– NIC) (Br

# Correção final de resíduos e geração de log

**Objetivo:**
Executar uma limpeza criteriosa dos arquivos finais de texto (.txt) que foram previamente organizados, eliminando linhas e trechos residuais resultantes da conversão de PDF ou erros comuns de digitalização. A etapa também registra todas as ocorrências em um log estruturado para posterior auditoria.

**O que o script faz:**
Percorre todas as subpastas de corpus_final, onde estão os textos separados por doença ou tema.

Em cada arquivo .txt, realiza as seguintes ações:

- Remove linhas contendo:

- links

- rodapés de documentos

- assinaturas eletrônicas

- datas soltas

- códigos técnicos

- sequências de símbolos ou pontos

- linhas com apenas números ou brancos

- Corrige pequenos resíduos (ex: espaços duplicados, travessões irregulares).

- Cria uma nova estrutura espelhada em textos_corrigidos, onde salva os arquivos processados.

- Gera um log detalhado em CSV, que inclui:

- Nome do arquivo

- Quantidade de linhas removidas

- Padrões suspeitos detectados

- Caminho do arquivo original

- Caminho do arquivo corrigido

In [28]:
# === CONFIGURAÇÃO DE CAMINHOS === #
PASTA_ORIGEM = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
PASTA_DESTINO = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos"
CAMINHO_LOG = os.path.join(PASTA_DESTINO, 'log_correcoes.csv')
os.makedirs(PASTA_DESTINO, exist_ok=True)

# === PADRÕES SUSPEITOS A REMOVER === #
PADROES_LINHAS_REMOVER = [
    r'https?:\/\/\S+',                                # links
    r'documento assinado eletronicamente',            # assinatura eletrônica
    r'nota técnica \d+',                              # rodapés duplicados
    r'código\s+crc\s+\w+',                            # códigos SEI
    r'pg\.?\s*\d+',                                   # "pg. 3"
    r'página\s+\d+\s+de\s+\d+',                       # "Página 3 de 12"
    r'^\s*$',                                         # linhas em branco
    r'^[\W_]{10,}$',                                  # linhas com só símbolos
    r'^[0-9]+[ \t]+[0-9]+$',                          # duas colunas numéricas soltas
    r'^\d{2}/\d{2}/\d{4}$',                           # datas soltas
    r'\.{4,}',                                        # sequência de pontos
]

# === FUNÇÃO DE LIMPEZA E LOG DE OCORRÊNCIAS === #
def limpar_texto(texto, arquivo_nome):
    linhas = texto.splitlines()
    linhas_corrigidas = []
    removidas = 0
    padroes_detectados = set()

    for linha in linhas:
        linha_original = linha
        for padrao in PADROES_LINHAS_REMOVER:
            if re.search(padrao, linha, re.IGNORECASE):
                padroes_detectados.add(padrao)
                linha = ''
                break

        linha = re.sub(r'[ \t]+', ' ', linha)
        linha = re.sub(r'–', '-', linha)
        linha = re.sub(r'_+', '_', linha)
        linha = linha.strip()
        if linha:
            linhas_corrigidas.append(linha)
        else:
            removidas += 1

    texto_corrigido = '\n'.join(linhas_corrigidas)
    return texto_corrigido, removidas, list(padroes_detectados)

# === CRIAR/ESCREVER O CSV DE LOG === #
with open(CAMINHO_LOG, 'w', newline='', encoding='utf-8') as log_file:
    writer = csv.writer(log_file)
    writer.writerow(['Arquivo', 'Linhas Removidas', 'Padrões Detectados', 'Original', 'Corrigido'])

    print("\n🔧 Iniciando correção e comparação...")

    for subpasta in os.listdir(PASTA_ORIGEM):
        caminho_sub = os.path.join(PASTA_ORIGEM, subpasta)
        if os.path.isdir(caminho_sub):
            nova_sub = os.path.join(PASTA_DESTINO, subpasta)
            os.makedirs(nova_sub, exist_ok=True)

            for arquivo in os.listdir(caminho_sub):
                if arquivo.endswith(".txt"):
                    caminho_arquivo = os.path.join(caminho_sub, arquivo)

                    with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as f:
                        original = f.read()

                    # Corrigir e registrar padrões encontrados
                    corrigido, linhas_removidas, padroes = limpar_texto(original, arquivo)

                    # Salvar texto corrigido
                    caminho_corrigido = os.path.join(nova_sub, arquivo)
                    with open(caminho_corrigido, 'w', encoding='utf-8') as f_out:
                        f_out.write(corrigido)

                    print(f"✅ Corrigido: {arquivo} | Linhas removidas: {linhas_removidas}")

                    # Escrever no log
                    writer.writerow([arquivo, linhas_removidas, '; '.join(padroes), caminho_arquivo, caminho_corrigido])

print("\n Log salvo em:", CAMINHO_LOG)
print(" Todos os textos foram processados e corrigidos.")


🔧 Iniciando correção e comparação...
✅ Corrigido: adenovirus_guia_1txt_guiavsa2023.txt | Linhas removidas: 244
✅ Corrigido: adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt | Linhas removidas: 244
✅ Corrigido: adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt | Linhas removidas: 244
✅ Corrigido: adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt | Linhas removidas: 244
✅ Corrigido: adenovírus_guia_1.txt | Linhas removidas: 244
✅ Corrigido: arboviroses_nota_tecnica_1txt_notatecnica2023.txt | Linhas removidas: 1
✅ Corrigido: arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023.txt | Linhas removidas: 1
✅ Corrigido: arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023.txt | Linhas removidas: 1
✅ Corrigido: arboviroses_nota_tecnica_1txt_notatecnica2023

# LanguageTool

Este script automatiza a limpeza e correção linguística de arquivos .txt utilizando o LanguageTool em servidor local, com integração via biblioteca language-tool-python.

O primeiro passo do script é garantir que a biblioteca language-tool-python esteja instalada. Em seguida, ele define os caminhos necessários para a execução: PASTA_ORIGEM (onde estão os textos brutos), PASTA_DESTINO (onde os textos corrigidos serão salvos), PASTA_LTX (pasta contendo o arquivo .jar, o .bat e o arquivo de porta do LanguageTool), PORTA_FILE (caminho até o arquivo porta_languagetool.txt que armazena a porta em uso) e BAT_FILE (arquivo .bat que inicia o servidor local do LanguageTool).

A função conectar_languagetool() é o núcleo do processo de conexão. Caso o arquivo de porta ainda não exista, o script executa automaticamente o .bat para iniciar o servidor. Depois, tenta ler a porta disponível no arquivo .txt e conecta-se ao servidor local (http://localhost:{porta}), realizando até 10 tentativas com intervalos de espera. Se não conseguir estabelecer a conexão, o script gera um erro indicando que não foi possível se conectar ao LanguageTool.

Esse processo garante que o servidor do LanguageTool esteja rodando antes de qualquer tentativa de uso do corretor ortográfico no restante do script, de forma robusta e automatizada.

**Requisitos obrigatórios:**
- Java instalado (Java 11 ou superior) – O LanguageTool requer Java para ser executado. É necessário verificar se o Java está instalado usando o comando java -version.

- Pasta do LanguageTool corretamente configurada – Essa pasta deve conter:

- languagetool-server.jar

- iniciar_languagetool.bat (para VS code é necessário inciar manualmente)

- porta_languagetool.txt

- Arquivo .bat com caminho correto – O iniciar_languagetool.bat deve conter o caminho real da pasta onde o .jar está localizado e ser capaz de identificar uma porta livre para inicializar o servidor. Ele grava automaticamente o número da porta no arquivo porta_languagetool.txt.

- Porta local disponível – O servidor será iniciado localmente na porta 8081 (ou na próxima disponível). É importante que não haja bloqueios no firewall que impeçam o uso da porta.

Se todos os requisitos estiverem atendidos, o script executa automaticamente todo o processo de inicialização do servidor e conexão ao corretor gramatical. Esse tipo de estrutura permite que novos arquivos sejam adicionados e processados de forma contínua, mantendo um fluxo de trabalho automatizado, eficiente e reutilizável.

Atenção: Se algum desses requisitos estiver ausente, o script apresentará erro de conexão e não conseguirá aplicar correções linguísticas.
java: https://www.oracle.com/java/technologies/javase/jdk17-archive-downloads.html
languagetool: https://languagetool.org/download/

In [29]:
# === CONFIGURAÇÕES DE CAMINHOS === #
PASTA_ORIGEM = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
PASTA_DESTINO = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos"
PASTA_LTX = r"C:\Users\isisi\Documents\IAVS_PROJETO\LanguageTool"
CAMINHO_LOG = os.path.join(PASTA_DESTINO, 'log_correcoes.csv')
PORTA_FILE = os.path.join(PASTA_LTX, "porta_languagetool.txt")
BAT_FILE = os.path.join(PASTA_LTX, "iniciar_languagetool.bat")

os.makedirs(PASTA_DESTINO, exist_ok=True)

# === TENTATIVA DE CONEXÃO COM LANGUAGETOOL === #
def conectar_languagetool():
    if not os.path.exists(PORTA_FILE):
        print("⚙️ Arquivo de porta não encontrado. Tentando iniciar o servidor LanguageTool...")
        subprocess.Popen(BAT_FILE, shell=True)
        time.sleep(5)  # espera o servidor levantar

    # Após tentar iniciar, tenta ler a porta
    for tentativa in range(10):
        if os.path.exists(PORTA_FILE):
            with open(PORTA_FILE, "r", encoding="utf-8") as f:
                porta = f.read().strip()
                print(f"🌐 Conectando ao LanguageTool na porta {porta}...")
                try:
                    return language_tool_python.LanguageTool('pt-BR', remote_server=f"http://localhost:{porta}")
                except Exception as e:
                    print(f"⏳ Tentativa {tentativa+1}/10 falhou: {e}")
                    time.sleep(3)
        else:
            time.sleep(2)

    raise RuntimeError("❌ Não foi possível iniciar ou conectar ao LanguageTool.")

# Conexão com LanguageTool
tool = conectar_languagetool()

🌐 Conectando ao LanguageTool na porta 8081...


# Visualização de Diferenças entre Textos Originais e Corrigidos
Este script foi desenvolvido para comparar visualmente os textos antes e depois da correção linguística, utilizando a biblioteca difflib do Python. A saída é um arquivo .html interativo que destaca as diferenças entre os textos, facilitando a validação do processo de correção.

**Objetivo**
Selecionar um arquivo aleatório por subpasta da pasta corpus_final, localizar sua versão corrigida em textos_corrigidos e gerar um arquivo HTML com as diferenças entre o original e o corrigido.

Organização de Pastas Esperada
corpus_final/ → Contém os arquivos .txt originais, organizados por subpastas (ex.: dengue, sarampo, etc.).

textos_corrigidos/ → Contém os arquivos corrigidos com a mesma estrutura de subpastas.

**O que o script faz**
Percorre todas as subpastas de corpus_final.

Em cada subpasta:

- Escolhe aleatoriamente um arquivo .txt.

- Localiza o arquivo correspondente na subpasta equivalente de textos_corrigidos.

- Gera um arquivo .html chamado comparacao_visual_<nome_do_arquivo>.html contendo:

- Texto original (à esquerda)

- Texto corrigido (à direita)

- Destaques coloridos das diferenças.

- O arquivo .html é salvo na mesma subpasta dentro de textos_corrigidos.

Requisitos
O script usa somente bibliotecas padrão do Python (os, random, pathlib, difflib), não requer instalação de pacotes adicionais.

Os arquivos .txt originais e corrigidos devem ter exatamente o mesmo nome e estarem organizados na mesma estrutura de pastas.

In [30]:
# -*- coding: utf-8 -*-
"""
Script para limpeza e correção linguística de arquivos .txt do projeto IAVS.
Somente novos arquivos ou arquivos ainda não processados são tratados.
Versão otimizada para velocidade e reaproveitamento de resultados anteriores.
"""
# === CONFIGURAÇÃO DE CAMINHOS === #
PASTA_ORIGEM = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
PASTA_DESTINO = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos"
CAMINHO_LOG = os.path.join(PASTA_DESTINO, 'log_correcoes.csv')
os.makedirs(PASTA_DESTINO, exist_ok=True)

# === CONFIGURAÇÃO DO CORRETOR ORTOGRÁFICO === #
tool = language_tool_python.LanguageTool('pt-BR', remote_server='http://localhost:8081')
MAX_CARACTERES = 20000  # Tamanho máximo por bloco (evita sobrecarregar API gratuita)

# === PADRÕES DE LINHAS A REMOVER === #
PADROES_LINHAS_REMOVER = [
    r'https?:\/\/\S+',                            # Links
    r'documento assinado eletronicamente',        # Assinatura eletrônica
    r'nota técnica \d+',                          # Rodapés
    r'código\s+crc\s+\w+',                        # Códigos SEI
    r'pg\.?\s*\d+',                               # Página abreviada
    r'página\s+\d+\s+de\s+\d+',                   # Página X de Y
    r'^\s*$',                                     # Linhas vazias
    r'^[\W_]{10,}$',                              # Somente símbolos
    r'^[0-9]+[ \t]+[0-9]+$',                      # Colunas numéricas
    r'^\d{2}/\d{2}/\d{4}$',                       # Datas soltas
    r'\.{4,}',                                    # Muitos pontos
]

# === FUNÇÃO PARA LIMPAR TEXTO === #
def limpar_texto(texto):
    linhas = texto.splitlines()
    linhas_corrigidas = []
    removidas = 0
    padroes_detectados = set()

    for linha in linhas:
        for padrao in PADROES_LINHAS_REMOVER:
            if re.search(padrao, linha, flags=re.IGNORECASE):
                padroes_detectados.add(padrao)
                linha = ''
                break

        linha = re.sub(r'[ \t]+', ' ', linha)
        linha = re.sub(r'–', '-', linha)
        linha = re.sub(r'_+', '_', linha)
        linha = linha.strip()

        if linha:
            linhas_corrigidas.append(linha)
        else:
            removidas += 1

    return '\n'.join(linhas_corrigidas), removidas, list(padroes_detectados)

# === FUNÇÃO PARA CORRIGIR ORTOGRAFIA EM BLOCOS === #
def aplicar_correcoes_linguisticas(texto):
    blocos_corrigidos = []
    for i in range(0, len(texto), MAX_CARACTERES):
        bloco = texto[i:i+MAX_CARACTERES]
        try:
            matches = tool.check(bloco)
            bloco_corrigido = language_tool_python.utils.correct(bloco, matches)
            blocos_corrigidos.append(bloco_corrigido)
        except Exception as e:
            print(f"⚠️ Erro ao corrigir bloco: {e}")
            blocos_corrigidos.append(bloco)
    return '\n'.join(blocos_corrigidos)

# === COLETA DE ARQUIVOS JÁ CORRIGIDOS === #
print("📂 Verificando arquivos já corrigidos...")
arquivos_corrigidos = set()
for subdir, _, arquivos in os.walk(PASTA_DESTINO):
    for arq in arquivos:
        if arq.endswith(".txt"):
            arquivos_corrigidos.add(arq)

# === ABRE O LOG DE CORREÇÃO PARA ESCREVER NOVOS REGISTROS === #
with open(CAMINHO_LOG, 'w', newline='', encoding='utf-8') as log_file:
    writer = csv.writer(log_file)
    writer.writerow(['Arquivo', 'Linhas Removidas', 'Padrões Detectados', 'Original', 'Corrigido'])

    print("🔧 Iniciando correção de novos arquivos...")

    for subpasta in os.listdir(PASTA_ORIGEM):
        caminho_sub = os.path.join(PASTA_ORIGEM, subpasta)
        if not os.path.isdir(caminho_sub):
            continue

        nova_sub = os.path.join(PASTA_DESTINO, subpasta)
        os.makedirs(nova_sub, exist_ok=True)

        for arquivo in os.listdir(caminho_sub):
            if not arquivo.endswith(".txt"):
                continue

            if arquivo in arquivos_corrigidos:
                print(f"⏩ {arquivo} já corrigido. Pulando.")
                continue

            caminho_arquivo = os.path.join(caminho_sub, arquivo)
            caminho_corrigido = os.path.join(nova_sub, arquivo)

            try:
                with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as f:
                    texto_original = f.read()

                texto_limpo, linhas_removidas, padroes = limpar_texto(texto_original)
                texto_corrigido = aplicar_correcoes_linguisticas(texto_limpo)

                with open(caminho_corrigido, 'w', encoding='utf-8') as f_out:
                    f_out.write(texto_corrigido)

                print(f"✅ {arquivo} | {linhas_removidas} linhas removidas | {len(texto_corrigido)} caracteres corrigidos")

                writer.writerow([
                    arquivo,
                    linhas_removidas,
                    '; '.join(padroes),
                    caminho_arquivo,
                    caminho_corrigido
                ])

            except Exception as e:
                print(f"❌ Erro ao processar {arquivo}: {e}")

print("\n✅ FIM! Log salvo em:", CAMINHO_LOG)
print("✅ Correção linguística e estrutural finalizada.")


📂 Verificando arquivos já corrigidos...
🔧 Iniciando correção de novos arquivos...
⏩ adenovirus_guia_1txt_guiavsa2023.txt já corrigido. Pulando.
⏩ adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt já corrigido. Pulando.
⏩ adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt já corrigido. Pulando.
⏩ adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt já corrigido. Pulando.
⏩ adenovírus_guia_1.txt já corrigido. Pulando.
⏩ arboviroses_nota_tecnica_1txt_notatecnica2023.txt já corrigido. Pulando.
⏩ arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023.txt já corrigido. Pulando.
⏩ arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023.txt já corrigido. Pulando.
⏩ arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023

**Correção linguística com LanguageTool**

**Objetivo:**
Executar a correção ortográfica e gramatical automatizada de todos os arquivos finais do projeto, integrando a verificação linguística do LanguageTool com a limpeza estrutural já aplicada anteriormente.

**O que essa etapa faz:**
- Conexão com o servidor local do LanguageTool:

- Lê a porta de conexão do arquivo porta_languagetool.txt.

- Se necessário, executa o arquivo .bat de inicialização do servidor.

- Limpeza estrutural dos arquivos:

- Remove links, rodapés, páginas, assinaturas eletrônicas, datas soltas, sequências de símbolos, entre outros ruídos.

- Uniformiza espaçamentos, travessões e underlines.

- Correção linguística com LanguageTool:

- Os arquivos são divididos em blocos de até 20.000 caracteres para não ultrapassar os limites do servidor.

- Cada bloco é verificado por erros de ortografia, concordância, pontuação e gramática.

- As sugestões do LanguageTool são aplicadas automaticamente.

- Salvamento dos arquivos corrigidos:

- Os textos corrigidos são armazenados em textos_corrigidos, em subpastas espelhadas.

**Registro em log:**
- Um arquivo CSV log_correcoes.csv armazena para cada arquivo:

- Quantidade de linhas removidas

- Padrões detectados

- Caminho original

- Caminho do texto corrigido

In [31]:
# === CONFIGURAÇÃO DE CAMINHOS === #
PASTA_ORIGEM = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
PASTA_DESTINO = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos"
CAMINHO_LOG = os.path.join(PASTA_DESTINO, 'log_correcoes.csv')
PORTA_FILE = r"C:\Users\isisi\Documents\IAVS_PROJETO\LanguageTool\porta_languagetool.txt"
BAT_FILE = r"C:\Users\isisi\Documents\LanguageTool\iniciar_languagetool.bat"

# Garante que a pasta de destino exista
os.makedirs(PASTA_DESTINO, exist_ok=True)

# === FUNÇÃO PARA CONECTAR AO SERVIDOR DO LANGUAGETOOL === #
def conectar_languagetool():
    if not os.path.exists(PORTA_FILE):
        print("⚠️ Arquivo de porta não encontrado.")
        if os.path.exists(BAT_FILE):
            print(" Tentando iniciar o servidor com o arquivo .bat...")
            subprocess.Popen(['start', '', BAT_FILE], shell=True)
            print(" Aguardando 10 segundos para o servidor iniciar...")
            time.sleep(10)
        else:
            raise FileNotFoundError(f"❌ Arquivo .bat não encontrado em: {BAT_FILE}")
    
    if os.path.exists(PORTA_FILE):
        with open(PORTA_FILE, "r", encoding="utf-8") as f:
            porta = f.read().strip()
        print(f"🌐 Conectando ao LanguageTool na porta {porta}...")
        return language_tool_python.LanguageTool('pt-BR', remote_server=f"http://localhost:{porta}")
    else:
        raise FileNotFoundError("❌ Falha ao localizar o arquivo da porta após tentativa de inicialização.")

# Estabelece conexão com o servidor local do LanguageTool
tool = conectar_languagetool()
MAX_CARACTERES = 20000  # Limite de caracteres por bloco a ser corrigido

# === PADRÕES SUSPEITOS A REMOVER === #
PADROES_LINHAS_REMOVER = [
    r'https?:\/\/\S+',
    r'documento assinado eletronicamente',
    r'nota técnica \d+',
    r'código\s+crc\s+\w+',
    r'pg\.?\s*\d+',
    r'página\s+\d+\s+de\s+\d+',
    r'^\s*$',
    r'^[\W_]{10,}$',
    r'^[0-9]+[ \t]+[0-9]+$',
    r'^\d{2}/\d{2}/\d{4}$',
    r'\.{4,}',
]

# === FUNÇÃO PARA LIMPAR RESÍDUOS TEXTUAIS === #
def limpar_texto(texto):
    linhas = texto.splitlines()
    linhas_corrigidas = []
    removidas = 0
    padroes_detectados = set()

    for linha in linhas:
        for padrao in PADROES_LINHAS_REMOVER:
            if re.search(padrao, linha, flags=re.IGNORECASE):
                padroes_detectados.add(padrao)
                linha = ''
                break
        linha = re.sub(r'[ \t]+', ' ', linha)
        linha = re.sub(r'–', '-', linha)
        linha = re.sub(r'_+', '_', linha)
        linha = linha.strip()

        if linha:
            linhas_corrigidas.append(linha)
        else:
            removidas += 1

    return '\n'.join(linhas_corrigidas), removidas, list(padroes_detectados)

# === FUNÇÃO PARA APLICAR CORREÇÕES LINGUÍSTICAS POR BLOCOS === #
def aplicar_correcoes_linguisticas(texto):
    blocos_corrigidos = []
    for i in range(0, len(texto), MAX_CARACTERES):
        bloco = texto[i:i+MAX_CARACTERES]
        try:
            matches = tool.check(bloco)
            bloco_corrigido = language_tool_python.utils.correct(bloco, matches)
            blocos_corrigidos.append(bloco_corrigido)
        except Exception as e:
            print(f"⚠️ Erro ao corrigir bloco: {e}")
            blocos_corrigidos.append(bloco)  # Se falhar, mantém o bloco original
    return '\n'.join(blocos_corrigidos)

# === EXECUÇÃO PRINCIPAL === #
start_time = time.time()

with open(CAMINHO_LOG, 'w', newline='', encoding='utf-8') as log_file:
    writer = csv.writer(log_file)
    writer.writerow(['Arquivo', 'Linhas Removidas', 'Padrões Detectados', 'Original', 'Corrigido'])

    print("\n🔧 Iniciando processamento com correção linguística...")

    for subpasta in os.listdir(PASTA_ORIGEM):
        caminho_sub = os.path.join(PASTA_ORIGEM, subpasta)
        if os.path.isdir(caminho_sub):
            nova_sub = os.path.join(PASTA_DESTINO, subpasta)
            os.makedirs(nova_sub, exist_ok=True)

            for arquivo in os.listdir(caminho_sub):
                if arquivo.endswith(".txt"):
                    caminho_arquivo = os.path.join(caminho_sub, arquivo)
                    caminho_corrigido = os.path.join(nova_sub, arquivo)

                    # Pula o arquivo se já estiver corrigido
                    if os.path.exists(caminho_corrigido):
                        print(f"⏩ Pulando (já corrigido): {arquivo}")
                        continue

                    # Leitura e limpeza estrutural
                    with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as f:
                        texto_original = f.read()
                    texto_limpo, linhas_removidas, padroes = limpar_texto(texto_original)

                    # Correção linguística com LanguageTool
                    texto_corrigido = aplicar_correcoes_linguisticas(texto_limpo)

                    # Salvando o novo texto corrigido
                    with open(caminho_corrigido, 'w', encoding='utf-8') as f_out:
                        f_out.write(texto_corrigido)

                    print(f"✅ {arquivo} | {linhas_removidas} linhas removidas | {len(texto_corrigido)} caracteres corrigidos")
                    writer.writerow([arquivo, linhas_removidas, '; '.join(padroes), caminho_arquivo, caminho_corrigido])

# === ENCERRAMENTO === #
tempo_total = time.time() - start_time
print(f"\n Tempo total de execução: {tempo_total:.2f} segundos")
print(" Log completo salvo em:", CAMINHO_LOG)
print(" Correção linguística e estrutural finalizada.")

🌐 Conectando ao LanguageTool na porta 8081...

🔧 Iniciando processamento com correção linguística...
⏩ Pulando (já corrigido): adenovirus_guia_1txt_guiavsa2023.txt
⏩ Pulando (já corrigido): adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt
⏩ Pulando (já corrigido): adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt
⏩ Pulando (já corrigido): adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.txt
⏩ Pulando (já corrigido): adenovírus_guia_1.txt
⏩ Pulando (já corrigido): arboviroses_nota_tecnica_1txt_notatecnica2023.txt
⏩ Pulando (já corrigido): arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023.txt
⏩ Pulando (já corrigido): arboviroses_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023txt_notatecnica2023.txt
⏩ Pulando (já corrigido): arboviroses_nota_tecnica_1txt_notatecnica2023txt_no

# Visualização de Diferenças entre Textos Originais e Corrigidos
Este script foi desenvolvido para comparar visualmente os textos antes e depois da correção linguística, utilizando a biblioteca difflib do Python. A saída é um arquivo .html interativo que destaca as diferenças entre os textos, facilitando a validação do processo de correção.

**Objetivo**
Selecionar um arquivo aleatório por subpasta da pasta corpus_final, localizar sua versão corrigida em textos_corrigidos e gerar um arquivo HTML com as diferenças entre o original e o corrigido.

**Organização de Pastas Esperada**
corpus_final/ → Contém os arquivos .txt originais, organizados por subpastas (ex.: dengue, sarampo, etc.).

textos_corrigidos/ → Contém os arquivos corrigidos com a mesma estrutura de subpastas.

**O que o script faz**
Percorre todas as subpastas de corpus_final.

**Em cada subpasta:**

- Escolhe aleatoriamente um arquivo .txt.

- ocaliza o arquivo correspondente na subpasta equivalente de textos_corrigidos.

- Gera um arquivo .html chamado comparacao_visual_<nome_do_arquivo>.html contendo:

- Texto original (à esquerda)

- Texto corrigido (à direita)

- Destaques coloridos das diferenças.

- O arquivo .html é salvo na mesma subpasta dentro de textos_corrigidos.

**Requisitos**
O script usa somente bibliotecas padrão do Python (os, random, pathlib, difflib), não requer instalação de pacotes adicionais.

Os arquivos .txt originais e corrigidos devem ter exatamente o mesmo nome e estarem organizados na mesma estrutura de pastas.


In [32]:
# === CONFIGURAÇÕES DE CAMINHO ===
PASTA_ORIGEM = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")
PASTA_CORRIGIDA = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos")

# === Função para comparar e salvar HTML ===
def gerar_diferencas_html(orig_path, corr_path, output_path):
    with open(orig_path, 'r', encoding='utf-8', errors='ignore') as f1:
        original = f1.readlines()
    with open(corr_path, 'r', encoding='utf-8', errors='ignore') as f2:
        corrigido = f2.readlines()

    d = difflib.HtmlDiff(wrapcolumn=120)
    html_diff = d.make_file(original, corrigido, fromdesc='Original', todesc='Corrigido')

    with open(output_path, 'w', encoding='utf-8') as out:
        out.write(html_diff)
    print(f"✅ Diferença salva em: {output_path}")

# === Loop por subpastas e seleção aleatória de um arquivo por pasta ===
for subdir, _, files in os.walk(PASTA_ORIGEM):
    rel_path = Path(subdir).relative_to(PASTA_ORIGEM)
    pasta_corrigida_sub = PASTA_CORRIGIDA / rel_path

    arquivos_txt = [f for f in files if f.endswith(".txt")]
    if arquivos_txt:
        nome_arquivo = random.choice(arquivos_txt)
        caminho_original = Path(subdir) / nome_arquivo
        caminho_corrigido = pasta_corrigida_sub / nome_arquivo

        if caminho_corrigido.exists():
            nome_saida = f"comparacao_visual_{nome_arquivo.replace('.txt', '')}.html"
            caminho_html = pasta_corrigida_sub / nome_saida
            gerar_diferencas_html(caminho_original, caminho_corrigido, caminho_html)
        else:
            print(f"⚠️ Arquivo corrigido não encontrado para: {nome_arquivo}")

✅ Diferença salva em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos\adenovírus\comparacao_visual_adenovirus_guia_1txt_guiavsa2023txt_guiavsa2023txt_guiavsa2023.html
✅ Diferença salva em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos\arboviroses\comparacao_visual_arboviroses_nota_técnica_1.html
✅ Diferença salva em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos\cancro\comparacao_visual_cancro_mole_protocolo_1txt_protocolo2023txt_protocolo2023.html
✅ Diferença salva em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos\chikungunya\comparacao_visual_chikungunya_protocolo_2txt_protocolo2023txt_protocolo2023.html
✅ Diferença salva em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos\clamídia\comparacao_visual_clamidia_protocolo_1txt_protocolo2023.html
✅ Diferença salva em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_corrigidos\coinfecção\comparacao_visual_coinfeccao_com_hepatite_b_nota_tecnica_1txt_notatecnica2023txt_notatecnica2023.html
✅ Diferen

# Geração de Relatório de Arquivos Textuais
Como etapa complementar ao armazenamento final dos arquivos, foi implementada a geração automática de um relatório-resumo em formato .csv. Esse relatório tem como objetivo fornecer uma visão geral da estrutura dos textos organizados por subpasta (doença ou tema), permitindo uma verificação quantitativa dos arquivos.

O que o relatório contém:
- Nome da subpasta: Classificação do documento (ex: dengue, zika, etc.).

- Nome do arquivo: Nome completo do arquivo .txt.

- Tamanho do arquivo (em bytes): Pode indicar se o arquivo está muito pequeno ou grande.

- Número de linhas: Pode sugerir a complexidade ou organização do texto.

- Número total de caracteres: Útil para identificar arquivos potencialmente vazios ou incompletos.

- O arquivo gerado, chamado resumo_corpus_final.csv, é salvo diretamente na pasta corpus_final.

Exemplo de saída:
| Subpasta    | Arquivo                       | Tamanho (bytes) | Nº de linhas | Nº de caracteres |
| ----------- | ----------------------------- | --------------- | ------------ | ---------------- |
| dengue      | dengue\_guiavsa2023.txt       | 12.304          | 455          | 11.982           |
| tuberculose | protocolo\_tb\_manual2023.txt | 18.734          | 672          | 17.401           |

In [33]:
# Caminho para o diretório com os textos finais organizados por subpasta
PASTA_FINAL = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"

# Caminho para salvar o relatório final
RELATORIO_CSV = os.path.join(PASTA_FINAL, "resumo_corpus_final.csv")

# Obtém a lista de subpastas no corpus final
subpastas = [nome for nome in os.listdir(PASTA_FINAL) if os.path.isdir(os.path.join(PASTA_FINAL, nome))]

# === Geração do relatório em CSV === #
with open(RELATORIO_CSV, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)

    # Cabeçalho do relatório
    writer.writerow(['Subpasta', 'Arquivo', 'Tamanho (bytes)', 'Nº de linhas', 'Nº de caracteres'])

    # Percorre cada subpasta
    for sub in subpastas:
        caminho_sub = os.path.join(PASTA_FINAL, sub)
        for arquivo in os.listdir(caminho_sub):
            if arquivo.endswith(".txt"):
                caminho_arquivo = os.path.join(caminho_sub, arquivo)

                # Lê o conteúdo do arquivo
                with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as arq:
                    conteudo = arq.read()
                    num_linhas = conteudo.count('\n')
                    num_chars = len(conteudo)
                    tamanho = os.path.getsize(caminho_arquivo)

                    # Escreve os dados no relatório
                    writer.writerow([sub, arquivo, tamanho, num_linhas, num_chars])

print(f" Relatório salvo em: {RELATORIO_CSV}")

 Relatório salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\resumo_corpus_final.csv
